<a href="https://colab.research.google.com/github/Nhatty1788/NatnaelMolago_DTSC3020_Fall2025/blob/main/assignment6_nsm0128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [32]:
 #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")

Dependencies installed.


### 2) Common Imports & Polite Headers

In [33]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")

Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [34]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    # Load all tables from the page
    from io import StringIO
    tables = pd.read_html(StringIO(html))

    # Return first table with >= 3 columns and flatten headers
    for t in tables:
        if t.shape[1] >= 3:
            return flatten_headers(t)

    # If no valid table found, return empty dataframe
    return pd.DataFrame()
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    # Standardize column names (handles space/underscore changes)
    df.columns = [c.strip().replace("_", " ") for c in df.columns]

    # strip whitespace
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

    # uppercase codes
    if "Alpha-2 code" in df.columns:
        df["Alpha-2 code"] = df["Alpha-2 code"].str.upper()
    if "Alpha-3 code" in df.columns:
        df["Alpha-3 code"] = df["Alpha-3 code"].str.upper()

    # numeric conversion
    if "Numeric code" in df.columns:
        df["Numeric code"] = pd.to_numeric(df["Numeric code"], errors="coerce").astype("Int64")

    # drop incomplete rows (only if column exists)
    cols = [c for c in ["Country", "Alpha-2 code", "Alpha-3 code"] if c in df.columns]
    df = df.dropna(subset=cols)

    return df
    raise NotImplementedError("TODO: implement q1_clean")


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    # Find the numeric column regardless of underscore/capitalization changes
    num_col = None
    for c in df.columns:
        if c.lower().replace("_", " ").strip() == "numeric code":
            num_col = c
            break

    if num_col is None:
        # If no numeric column found, return top rows as-is
        return df.head(top)

    # Sort by detected numeric column
    return df.sort_values(num_col, ascending=False).head(top)
    raise NotImplementedError("TODO: implement q1_sort_top")

In [50]:
url = "https://www.iban.com/country-codes"
html = fetch_html(url)
df_raw = q1_read_table(html)
df_clean = q1_clean(df_raw)
df_top15 = q1_sort_top(df_clean)

print(df_top15)
df_clean.to_csv("data_q1.csv", index=False)

                Country Alpha-2 code Alpha-3 code  Numeric
0           Afghanistan           AF          AFG        4
1         Åland Islands           AX          ALA      248
2               Albania           AL          ALB        8
3               Algeria           DZ          DZA       12
4        American Samoa           AS          ASM       16
5               Andorra           AD          AND       20
6                Angola           AO          AGO       24
7              Anguilla           AI          AIA      660
8            Antarctica           AQ          ATA       10
9   Antigua and Barbuda           AG          ATG       28
10            Argentina           AR          ARG       32
11              Armenia           AM          ARM       51
12                Aruba           AW          ABW      533
13            Australia           AU          AUS       36
14              Austria           AT          AUT       40


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [103]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    soup = BeautifulSoup(html, "lxml")
    items = soup.select(".athing")

    rows = []
    for item in items:
        rank_tag = item.select_one(".rank")
        title_tag = item.select_one(".titleline a")

        # subtext is in next <tr>
        sub = item.find_next_sibling("tr").select_one(".subtext")

        points = sub.select_one(".score") if sub else None
        user = sub.select_one(".hnuser") if sub else None

        # comments link = last <a> in subtext
        comments_link = sub.find_all("a")[-1] if sub else None
        comments_text = comments_link.text if comments_link else ""

        rows.append({
            "rank": rank_tag.text.replace(".", "") if rank_tag else "",
            "title": title_tag.text if title_tag else "",
            "link": title_tag["href"] if title_tag else "",
            "points": points.text.replace(" points", "").replace(" point", "") if points else "",
            "comments": comments_text.replace(" comments", "").replace(" comment", "") if comments_text else "",
            "user": user.text if user else ""
        })

    df = pd.DataFrame(rows)
    return df
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    # numeric fields → int or 0
    for col in ["rank", "points", "comments"]:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    # fill blank strings
    for col in ["title", "link", "user"]:
        df[col] = df[col].fillna("").astype(str)

    return df
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    return df.sort_values("points", ascending=False).head(top)
    raise NotImplementedError("TODO: implement q2_sort_top")

In [112]:
# Q2 — Write your answer here

url = "https://news.ycombinator.com/"
html = fetch_html(url)

df_raw = q2_parse_items(html)
df_clean = q2_clean(df_raw)
df_top15 = q2_sort_top(df_clean)

print(df_top15)
df_clean.to_csv("data_q2.csv", index=False)

    rank                                              title  \
8      9                           Leaving Meta and PyTorch   
12    13                                    A Fond Farewell   
26    27  Meta projected 10% of 2024 revenue came from s...   
1      2  Vodafone Germany is killing the open internet ...   
13    14  Denmark's government aims to ban access to soc...   
0      1  Rockstar employee shares account of the compan...   
6      7                                       I Love OCaml   
23    24  OpenMW 0.50.0 Released – open-source Morrowind...   
9     10                       Gmail AI gets more intrusive   
28    29             Text case changes the size of QR codes   
29    30                 We chose OCaml to write Stategraph   
19    20                                     PyTorch Helion   
20    21  Toxic Salton Sea dust triggers changes in lung...   
7      8  James Watson, Co-Discoverer of the Structure o...   
2      3  Myna: Monospace typeface designed for symbol-